<a href="https://colab.research.google.com/github/Yanina-Kutovaya/GNN/blob/main/notebooks/1_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Реализация Graph Convolutional Network (GCN) из PyTorch Geometric для классификации узлов на датасете Bitcoin-OTC с обработкой признаков рёбер

Задача — предсказать "уровень доверия" пользователей

Признаки узлов/рёбер - синтетические

Необходимо выбрать среду выполнения с GPU: Runtime → Change runtime type → GPU

## 1. Установка зависимостей

In [1]:
#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.8 MB/s eta 0:00:00


## 2. Импорт библиотек

In [2]:
import numpy as np

import torch
import torch.nn.functional as F
from torch_geometric.datasets import BitcoinOTC
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GCNConv

## 3. Загрузка и подготовка данных

Датасет Bitcoin-OTC:
* 5881 узел (пользователи)
* Рёбра содержат признак edge_attr (уровень доверия от -10 до 10)
* Синтетические признаки узлов: нормализованная степень узла
* Синтетические метки: 3 класса на основе активности пользователей

In [3]:
dataset = BitcoinOTC(root='data/BitcoinOTC', edge_window_size=10)
data = dataset[0]

Extracting data/BitcoinOTC/raw/soc-sign-bitcoinotc.csv.gz
Processing...
Done!


### 3.1 Создание синтетических признаков узлов (степень узла)

In [4]:
degrees = np.zeros(data.num_nodes)
for edge in data.edge_index.t().tolist():
    degrees[edge[0]] += 1
data.x = torch.tensor(degrees, dtype=torch.float).view(-1, 1)

### 3.2 Нормализация признаков

In [5]:
data.x = (data.x - data.x.mean()) / data.x.std()

### 3.3 Создание синтетических меток (3 класса на основе квантилей степени)

In [6]:
labels = torch.zeros(data.num_nodes, dtype=torch.long)
q1 = np.quantile(degrees, 0.33)
q2 = np.quantile(degrees, 0.66)
labels[degrees > q2] = 2
labels[(degrees > q1) & (degrees <= q2)] = 1
data.y = labels.to(torch.long)

### 3.4 Разделение данных на train/val/test

In [7]:
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

indices = torch.randperm(data.num_nodes)
data.train_mask[indices[:int(0.6*data.num_nodes)]] = True
data.val_mask[indices[int(0.6*data.num_nodes):int(0.8*data.num_nodes)]] = True
data.test_mask[indices[int(0.8*data.num_nodes):]] = True

## 4. Определение модели GCN с обработкой рёбер

Архитектура модели:
* Два GCN-слоя с использованием весов рёбер (edge_weight)
* Dropout для регуляризации
* Функция потерь: CrossEntropy

In [8]:
class EdgeAwareGCN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, x, edge_index, edge_attr):
        # Явное преобразование типов
        x = x.to(torch.float32)
        edge_attr = edge_attr.to(torch.float32)

        x = self.conv1(x, edge_index, edge_weight=edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        return self.conv2(x, edge_index, edge_weight=edge_attr)

## 5. Инициализация модели

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgeAwareGCN(num_classes=3).to(device)

data.x = data.x.to(torch.float32)
data.edge_attr = data.edge_attr.to(torch.float32)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

## 6. Обучение

In [10]:
def accuracy(pred, true, mask):
    return (pred[mask].argmax(dim=1) == true[mask]).sum().float() / mask.sum()

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Валидация
    model.eval()
    with torch.no_grad():
        val_acc = accuracy(out, data.y, data.val_mask)

    if epoch % 20 == 0:
        print(f'Epoch {epoch:03d} | Loss: {loss.item():.4f} | Val Acc: {val_acc:.4f}')

Epoch 000 | Loss: 1.1168 | Val Acc: 0.0616
Epoch 020 | Loss: 0.4567 | Val Acc: 0.9992
Epoch 040 | Loss: 0.0659 | Val Acc: 0.9992
Epoch 060 | Loss: 0.0166 | Val Acc: 0.9992
Epoch 080 | Loss: 0.0097 | Val Acc: 0.9992
Epoch 100 | Loss: 0.0075 | Val Acc: 0.9992
Epoch 120 | Loss: 0.0063 | Val Acc: 0.9992
Epoch 140 | Loss: 0.0056 | Val Acc: 0.9992
Epoch 160 | Loss: 0.0051 | Val Acc: 0.9992
Epoch 180 | Loss: 0.0048 | Val Acc: 0.9992


## 7. Тестирование

In [11]:
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index, data.edge_attr)
    test_acc = accuracy(out, data.y, data.test_mask)
    print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.9975
